In [1]:
import tensorflow as tf
import pydicom as dicom
import glob
import matplotlib.pyplot as plt
import os
import nibabel as nib
from skimage.io import imsave
import numpy as np
from tensorflow.data.experimental import sample_from_datasets
from tensorflow.image import central_crop
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from tensorflow.keras import Model
from skimage.util import crop
from tensorflow.keras.optimizers import *
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from tensorflow.keras import metrics

In [104]:
keras.image_data_format()

'channels_last'

In [2]:
def reshaping(array):
    array = tf.expand_dims(array, axis = -1)
    return(array)

In [3]:
def get_folders():
    """
    Returns a list of all the folders in the HGG and LGG directories.
    """
    HGG_Folders= os.listdir(r'C:\Users\Joe Krinke\Desktop\BraTS 2019 Data\MICCAI_BraTS_2019_Data_Training\HGG')
    LGG_Folders = os.listdir(r'C:\Users\Joe Krinke\Desktop\BraTS 2019 Data\MICCAI_BraTS_2019_Data_Training\LGG')
    return(HGG_Folders, LGG_Folders)

In [93]:
def load_nii_data(filename):
    """
    Load in an nii file and return an array containing the pixel data. 
    inputs:
        filename (string): The name of the nii file you want to load.
    outputs:
        array (numpy array): A numpy array of the given image.
    """
    image = nib.load(filename)
    array = image.get_fdata()
    array = array.astype(np.uint8)
    return(array)

In [5]:
def crop(img, start, end):
    img = img[start:end, start:end, :]
    return(img)

In [6]:
def feature_scale(img):
    """
    Scale the features of the image from 0 to 1
    """
    img *= 255.0/img.max()  
    return (img)

In [7]:
def voxel_clip(img):
    """
    Clips the image to the 2nd and 98th percentile values.
    inputs:
        img (a numpy array): The image you want to clip
    outputs:
        img (a numpy array): The image with its values clipped
    """
    upper = np.percentile(img, 98)
    lower = np.percentile(img, 2)
    img[img > upper] = upper
    img[img < lower] = lower
    return(img)

In [71]:
def convert_bool(array):
    array = np.where(array>0, 1, 0)
    return(array)

In [167]:
def LGG_data_generator():
    """
    Create dataset of LGG data.
    inputs:
        path (string): The path where the folders containing the images are located.
    outputs:
        Yields image, mask tuples. Each mask appears with its corresponding image. Masks appear multiple times (once for each image type: t1,t2, etc.).
        
    """
    path = r'C:/Users/Joe Krinke/Desktop/BraTS 2019 Data/MICCAI_BraTS_2019_Data_Training/'
    image_types = ['flair', 't1', 't2', 't1ce']
    HGG_Folders, LGG_Folders = get_folders()
                         
    for i in LGG_Folders:
        mask = load_nii_data(path + 'LGG/' + i + '/' + i + '_' + 'seg' + '.nii.gz')
        # Crop Mask
        mask = crop(mask, 0,224)
        for types in image_types:
            # crop image, clip it
            data = load_nii_data(path + 'LGG/' + i + '/' + i + '_' + types + '.nii.gz')
            data = crop(data, 0, 224)
            data = voxel_clip(data)
            img_counter = 0
            for j in range(data.shape[2]):
                img_counter += 1 
                yield(reshaping(data[:,:,j]), reshaping(convert_bool(mask[:,:,j])))


def HGG_data_generator():
    """ 
    Create dataset of HGG data.
    inputs:
        path (string): The path where the folders containing the images are located.
    outputs:
        Yields image, mask tuples. Each mask appears with its corresponding image. Masks appear multiple times (once for each image type: t1,t2, etc.).
        
    """
    path = r'C:/Users/Joe Krinke/Desktop/BraTS 2019 Data/MICCAI_BraTS_2019_Data_Training/'
    image_types = ['flair', 't1', 't2', 't1ce']
    HGG_Folders, LGG_Folders = get_folders()
    for i in HGG_Folders:
        mask = load_nii_data(path + 'HGG/' + i + '/' + i + '_' + 'seg' + '.nii.gz')
        # Crop Mask
        mask = crop(mask, 0, 224)
        for types in image_types:
            data = load_nii_data(path + 'HGG/'+ i + '/'+ i + '_' + types + '.nii.gz')
            # crop image, clip it
            data = crop(data, 0, 224)
            data = voxel_clip(data)
            img_counter = 0
            for j in range(data.shape[2]):
                img_counter += 1 
                yield(reshaping(data[:,:,j]), reshaping(convert_bool(mask[:,:,j])))

In [168]:
# Create both datasets
HGG_dataset = tf.data.Dataset.from_generator(HGG_data_generator, output_types = (tf.uint8, tf.uint8), output_shapes = (tf.TensorShape([224,224,1]), tf.TensorShape([224,224,1])))
LGG_dataset = tf.data.Dataset.from_generator(LGG_data_generator, output_types = (tf.uint8,tf.uint8), output_shapes = (tf.TensorShape([224,224,1]), tf.TensorShape([224,224,1])))

In [169]:
# Create master dataset with examples from both datasets
full_dataset = tf.data.experimental.sample_from_datasets([HGG_dataset, LGG_dataset], seed =123)

In [139]:
def unet(input_size = (224,224,1)):
    ''' 
    U-Net model.
    Inputs:
        input_size(tuple):  Model input size
    '''

    inputs = Input(input_size)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same',
                kernel_initializer=initializers.random_normal(stddev=0.01))(conv5),conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',
                kernel_initializer=initializers.random_normal(stddev=0.01))(conv6),conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2),padding='same',
                kernel_initializer=initializers.random_normal(stddev=0.01))(conv7),conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',)(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same',
                kernel_initializer=initializers.random_normal(stddev=0.01))(conv8),conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',
                   kernel_initializer=initializers.random_normal(stddev=0.01))(conv9)

    conv10 = Conv2D(1, (1, 1), activation='relu',
                    kernel_initializer=initializers.random_normal(stddev=0.01))(conv9)
    conv10 = Activation('sigmoid')(conv10)
    model = Model(inputs=[inputs], outputs=[conv10])

    lr = 1e-5
    model.compile(optimizer=Adam(lr=lr), loss='BinaryCrossentropy', metrics=['BinaryCrossentropy'])
    return(model)

In [140]:
test_model = unet()

In [141]:
test_model.summary()

Model: "functional_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d_304 (Conv2D)             (None, 224, 224, 32) 320         input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_305 (Conv2D)             (None, 224, 224, 32) 9248        conv2d_304[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_64 (MaxPooling2D) (None, 112, 112, 32) 0           conv2d_305[0][0]                 
______________________________________________________________________________________

In [16]:
# Get data train and val size
data_size = 0
for element in full_dataset:
    data_size += 1
train_size = int(0.80* data_size)
val_size = int(0.20 * data_size)

In [118]:
# Create training and validation datasets
train_dataset = full_dataset.take(train_size)
val_dataset = full_dataset.skip(val_size)

In [126]:
batch_train = train_dataset.batch(4)

In [120]:
batch_train

<BatchDataset shapes: ((None, 224, 224, 1), (None, 224, 224, 1)), types: (tf.uint8, tf.uint8)>

In [121]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [122]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [142]:
test_model.fit(batch_train,epochs=20, verbose=1, callbacks=[callback])

Epoch 1/20
    263/Unknown - 52s 197ms/step - loss: 0.6931 - binary_crossentropy: 0.6931

KeyboardInterrupt: 

Investigate dataset